# Bidirectionnal Prediction

In [70]:
from keras.layers import Dense, Reshape, Input, GRU, Embedding, RepeatVector, Bidirectional, Concatenate, LSTM
from keras.models import load_model, Model
from keras.utils import Sequence

import matplotlib.pyplot as plt
import csv
import math
import numpy as np

import tensorflow as tf

## Bidirectionnal without context

In [ ]:
def BidirModel():
  inpt = Input(shape=(34 , 2))
  x = Bidirectional(GRU(32 , return_sequences=True))(inpt)
  x = Bidirectional(GRU(32 , return_sequences=True))(inpt)
  x = Dense(2)(x)
  model = Model(inputs=inpt , outputs=x)

  return model

model = BidirModel()
model.compile(loss="mean_squared_error" , optimizer="adam")
model.summary()
#model.save("/content/drive/MyDrive/Hackathon IA/Models/Prediction/Bidir_save")

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 34, 2)]           0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 34, 64)            6912      
_________________________________________________________________
dense_20 (Dense)             (None, 34, 2)             130       
Total params: 7,042
Trainable params: 7,042
Non-trainable params: 0
_________________________________________________________________


In [2]:
X = []
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/X_train.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1:
      x = []
      for j in range(34):
        x.append([int(row[2*j]) , int(row[2*j+1])])
      X.append(x)

X = np.array(X)
print(X.shape)

(234643, 34, 2)


In [3]:
Y = []
i = 0
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/Y_train.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1 and i < 234643:
      y = []
      for j in range(34):
        y.append([int(row[2*j]) , int(row[2*j+1])])
      Y.append(y)
      i+=1

Y = np.array(Y) - X
print(Y.shape)

(234643, 34, 2)


In [ ]:
#model = load_model("/content/drive/MyDrive/Hackathon IA/Models/Prediction/Bidir2_save")
model.fit(X , Y , batch_size=4096, epochs=100, shuffle=True)
#model.save("/content/drive/MyDrive/Hackathon IA/Models/Prediction/Bidir_save")

Epoch 1/100
58/58 [==============================] - 6s 45ms/step - loss: 4.1992
Epoch 2/100
58/58 [==============================] - 3s 43ms/step - loss: 3.9850
Epoch 3/100
58/58 [==============================] - 3s 43ms/step - loss: 3.9421
Epoch 4/100
58/58 [==============================] - 2s 43ms/step - loss: 3.9098
Epoch 5/100
58/58 [==============================] - 2s 43ms/step - loss: 3.8787
Epoch 6/100
58/58 [==============================] - 2s 43ms/step - loss: 3.8521
Epoch 7/100
58/58 [==============================] - 2s 43ms/step - loss: 3.8305
Epoch 8/100
58/58 [==============================] - 3s 43ms/step - loss: 3.8011
Epoch 9/100
58/58 [==============================] - 2s 43ms/step - loss: 3.7954
Epoch 10/100
58/58 [==============================] - 2s 43ms/step - loss: 3.7843
Epoch 11/100
58/58 [==============================] - 2s 43ms/step - loss: 3.7784
Epoch 12/100
58/58 [==============================] - 2s 43ms/step - loss: 3.7682
Epoch 13/100
58/58 [=====

In [4]:
X_val = []
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/X_dev.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1:
      x = []
      for j in range(34):
        x.append([int(row[2*j]) , int(row[2*j+1])])
      X_val.append(x)

X_val = np.array(X_val)
print(X_val.shape)

(13500, 34, 2)


In [5]:
Y_val = []
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/Y_dev.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1:
      x = []
      for j in range(34):
        x.append([int(row[2*j]) , int(row[2*j+1])])
      Y_val.append(x)

Y_val = np.array(Y_val) - X_val
print(Y_val.shape)

(13500, 34, 2)


In [ ]:
model.evaluate(X_val , Y_val , batch_size=4096)

4/4 [==============================] - 1s 17ms/step - loss: 3.5337


3.5337047576904297

**Uncentered :**

| Model | Train Loss | Val Loss |
|-------|-------|--------|
| Bidir GRU 32 | 3.7003 | 3.6086 |
| Bidir LSTM 32 | 3.7178 | 3.6179 |
| Bidir GRU 64 | 3.1995 | 3.6180 |
| Bidir 2-GRU 32-8 | 3.5840 | 3.5874 |
| Bidir 2-GRU 32-32 | 3.2392  | 3.5427 |
| Bidir 2-GRU 64-64 | 2.4759 | 3.9949 |

**Centered data :**

| Model | Train Loss | Val Loss |
|-------|-------|--------|
| Bidir GRU 32 | 3.4861 | 3.5446 |
| Bidir 2-GRU 32-8 | 3.7539 | 3.6333 |
| Bidir 2-GRU 32-32 | 3.4748  | 3.5337 |


## Bidirectionnal with context

In [ ]:
def BidirModelContext():
  inpt1 = Input(shape=(34 , 2))
  inpt2 = Input(shape=(24))
  context = Dense(12)(inpt2)
  context = Dense(2)(context)
  context = RepeatVector(34)(context)

  x = Concatenate(axis=-1)([inpt1 , context])

  x = Bidirectional(GRU(32 , return_sequences=True))(x)
  x = Bidirectional(GRU(64 , return_sequences=True))(x)
  
  x = Dense(2)(x)
  model = Model(inputs=[inpt1 ,inpt2] , outputs=x)

  return model

model = BidirModelContext()
model.compile(loss="mean_squared_error" , optimizer="adam")
model.summary()
#model.save("/content/drive/MyDrive/Hackathon IA/Models/Prediction/BidirContext_save")

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 24)]         0                                            
__________________________________________________________________________________________________
dense_21 (Dense)                (None, 12)           300         input_17[0][0]                   
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 2)            26          dense_21[0][0]                   
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 34, 2)]      0                                            
____________________________________________________________________________________________

In [26]:
Context = []
i = 0
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/Context_train.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1 and i < 234643:
      Context.append([float(i) for i in row])
      i+=1

Context = np.array(Context)
print(Context.shape)

(234643, 24)


In [ ]:
model.fit([X , Context] , Y, batch_size=8192, epochs=20, verbose = 2)

Epoch 1/20
29/29 - 6s - loss: 3.5779
Epoch 2/20
29/29 - 6s - loss: 3.5582
Epoch 3/20
29/29 - 6s - loss: 3.5484
Epoch 4/20
29/29 - 6s - loss: 3.5398
Epoch 5/20
29/29 - 6s - loss: 3.5378
Epoch 6/20
29/29 - 6s - loss: 3.5170
Epoch 7/20
29/29 - 6s - loss: 3.5014
Epoch 8/20
29/29 - 6s - loss: 3.4935
Epoch 9/20
29/29 - 6s - loss: 3.4859
Epoch 10/20
29/29 - 6s - loss: 3.4818
Epoch 11/20
29/29 - 6s - loss: 3.4817
Epoch 12/20
29/29 - 6s - loss: 3.4531
Epoch 13/20
29/29 - 6s - loss: 3.4589
Epoch 14/20
29/29 - 6s - loss: 3.4336
Epoch 15/20
29/29 - 6s - loss: 3.4184
Epoch 16/20
29/29 - 6s - loss: 3.4070
Epoch 17/20
29/29 - 6s - loss: 3.4105
Epoch 18/20
29/29 - 6s - loss: 3.4035
Epoch 19/20
29/29 - 6s - loss: 3.3817
Epoch 20/20
29/29 - 6s - loss: 3.3693


In [ ]:
#model.save("/content/drive/MyDrive/Hackathon IA/Models/Prediction/BidirContext_save")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Hackathon IA/Models/Prediction/BidirContext_save/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Hackathon IA/Models/Prediction/BidirContext_save/assets


In [27]:
Context_val = []
i = 0
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/Context_dev.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1 and i < 234643:
      y = [float(i) for i in row]
      Context_val.append(y)
      i+=1

Context_val = np.array(Context_val)
print(Context_val.shape)

(13500, 24)


In [ ]:
model.evaluate([X_val , Context_val] , Y_val, batch_size=4096)

4/4 [==============================] - 0s 49ms/step - loss: 3.6404


3.640357732772827

| Model | Train Loss | Val Loss |
|-------|-------|--------|
| Bidir GRU 64 C-12-2-Sum | 3.3397 | 3.6230 |
| Bidir GRU 64 C-2-Sum | 3.7265 | 3.6934 |
| Bidir GRU 32 C-12-2-Concat | 3.7916 | 3.6721 |
| Bidir 2-GRU 32-32 C-12-2-Concat | 3.6107 | 3.5724 |
| Bidir 2-GRU 32-64 C-12-2-Concat | 3.3464 | 3.5382 |
| Bidir 2-GRU 64-64 C-12-8-Concat | 3.4705 | 3.5506 |

**Centered Data :**

| Model | Train Loss | Val Loss |
|-------|-------|--------|
| Bidir GRU 64 C-12-2-Sum | 3.3397 | 3.6230 |



## Bidirectionnal with station Embedding

In [6]:
embedd = load_model("/content/drive/MyDrive/Hackathon IA/Models/Data Mining/Embed256DModel").get_layer(index=0).get_weights()[0]
embedd.shape
embedd = np.array([embedd.copy() for i in range(234643)])

In [18]:
def EmbeddModel():
  inpt1 = Input(shape=(34 , 2))
  fixed_input = Input(shape=(34,8))
  context = Dense(32)(fixed_input)
  context = Dense(8)(context)

  x = Concatenate(axis=-1)([inpt1 , context])

  x = Bidirectional(GRU(16 , return_sequences=True))(x)  
  x = Bidirectional(GRU(16 , return_sequences=True))(x)  
  x = Dense(2)(x)

  model = Model(inputs=[inpt1 , fixed_input] , outputs=x)

  return model

model = EmbeddModel()
model.compile(loss="mean_squared_error" , optimizer="adam")
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 34, 8)]      0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 34, 32)       288         input_8[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 34, 2)]      0                                            
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 34, 8)        264         dense_9[0][0]                    
____________________________________________________________________________________________

In [23]:
model.fit([X,embedd] , Y, batch_size=4096, epochs=50, verbose = 1)

Epoch 1/50
58/58 [==============================] - 2s 32ms/step - loss: 3.2933
Epoch 2/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2918
Epoch 3/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2863
Epoch 4/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2882
Epoch 5/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2823
Epoch 6/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2814
Epoch 7/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2802
Epoch 8/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2781
Epoch 9/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2790
Epoch 10/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2771
Epoch 11/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2757
Epoch 12/50
58/58 [==============================] - 2s 31ms/step - loss: 3.2737
Epoch 13/50
58/58 [==================

In [7]:
embedd_val = load_model("/content/drive/MyDrive/Hackathon IA/Models/Data Mining/Embed256DModel").get_layer(index=0).get_weights()[0]
embedd_val.shape
embedd_val = np.array([embedd_val.copy() for i in range(13500)])

In [24]:
model.evaluate([X_val,embedd_val] , Y_val, batch_size=4096, verbose = 1)

4/4 [==============================] - 0s 14ms/step - loss: 3.5177


3.5177063941955566

| Model | Train Loss | Val Loss |
|-------|-------|--------|
| Bidir GRU 32 E-32-8-Concat | 3.2586 | 3.5954 |
| Bidir 2-GRU 32-32 E-32-8-Concat | 2.7857 | 3.8604 |
| Bidir 2-GRU 16-32 E-32-8-Concat ES| 3.3424 | 3.5275 |
| Bidir 2-GRU 16-16 E-32-8-Concat ES| 3.4263 | 3.4800 |
| Bidir 2-GRU 16-16 E-32-8-Concat| 3.2912 | 3.4306 |


## Feed Everything In !

In [49]:
def FeedEvIn():  
  inpt2 = Input(shape=(34,8))
  context = Dense(32)(inpt2)
  context = Dense(8)(context)

  inpt1 = Input(shape=(34 , 2))
  x = Concatenate(axis=-1)([inpt1 , context])

  inpt3 = Input(shape=(24))
  context2 = Dense(12)(inpt3)
  context2 = Dense(2)(context2)
  context2 = RepeatVector(34)(context2)

  x = Concatenate(axis=-1)([x , context2])

  x = Bidirectional(GRU(16 , return_sequences=True))(x)  
  x = Bidirectional(GRU(32 , return_sequences=True))(x)  
  x = Dense(2)(x)

  model = Model(inputs=[inpt1 , inpt2, inpt3] , outputs=x)

  return model

model = FeedEvIn()
model.compile(loss="mean_squared_error" , optimizer="adam")
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, 34, 8)]      0                                            
__________________________________________________________________________________________________
input_39 (InputLayer)           [(None, 24)]         0                                            
__________________________________________________________________________________________________
dense_56 (Dense)                (None, 34, 32)       288         input_37[0][0]                   
__________________________________________________________________________________________________
dense_58 (Dense)                (None, 12)           300         input_39[0][0]                   
___________________________________________________________________________________________

In [52]:
model.fit([X , embedd, Context] , Y, batch_size=4096, verbose=1, epochs=50)

Epoch 1/50
58/58 [==============================] - 2s 41ms/step - loss: 3.5605
Epoch 2/50
58/58 [==============================] - 2s 39ms/step - loss: 3.5547
Epoch 3/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5513
Epoch 4/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5480
Epoch 5/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5475
Epoch 6/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5380
Epoch 7/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5303
Epoch 8/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5282
Epoch 9/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5245
Epoch 10/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5188
Epoch 11/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5114
Epoch 12/50
58/58 [==============================] - 2s 40ms/step - loss: 3.5061
Epoch 13/50
58/58 [==================

In [55]:
model.evaluate([X_val , embedd_val, Context_val] , Y_val, batch_size=4096, verbose=1)

4/4 [==============================] - 0s 17ms/step - loss: 3.5238


3.5237932205200195

| Model | Train Loss | Val Loss |
|-------|-------|--------|
| Bidir GRU 24-16 E-32-8-Concat C-12-2-Concat | 3.4619 | 3.4993 |
| Bidir GRU 32-16 E-32-8-Concat C-12-2-Concat | 3.3319 | 3.5235 |

## Sorting the stations according to CD

In [56]:
sortorder = [0,
 31,
 13,
 12,
 11,
 10,
 9,
 8,
 33,
 6,
 5,
 4,
 3,
 2,
 1,
 7,
 30,
 29,
 28,
 27,
 26,
 25,
 24,
 21,
 22,
 20,
 19,
 18,
 17,
 32,
 15,
 14,
 23,
 16]

In [57]:
X_sort = []
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/X_train.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1:
      x = []
      for j in sortorder:
        x.append([int(row[2*j]) , int(row[2*j+1])])
      X_sort.append(x)

X_sort = np.array(X_sort)
print(X_sort.shape)

(234643, 34, 2)


In [62]:
Y_sort = []
i = 0
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/Y_train.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1 and i < 234643:
      y = []
      for j in sortorder:
        y.append([int(row[2*j]) , int(row[2*j+1])])
      Y_sort.append(y)
      i+=1

Y_sort = np.array(Y_sort) - X_sort
print(Y_sort.shape)

(234643, 34, 2)


In [60]:
X_sort_val = []
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/X_dev.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1:
      x = []
      for j in sortorder:
        x.append([int(row[2*j]) , int(row[2*j+1])])
      X_sort_val.append(x)

X_sort_val = np.array(X_sort_val)
print(X_sort_val.shape)

(13500, 34, 2)


In [65]:
Y_sort_val = []
i = 0
with open("/content/drive/MyDrive/Hackathon IA/Datasets/San Francisco/Y_dev.csv" , "r") as file:
  spamreader = csv.reader(file)
  for row in spamreader:
    if len(row) > 1 and i < 234643:
      y = []
      for j in sortorder:
        y.append([int(row[2*j]) , int(row[2*j+1])])
      Y_sort_val.append(y)
      i+=1

Y_sort_val = np.array(Y_sort_val) - X_sort_val
print(Y_sort_val.shape)

(13500, 34, 2)


In [79]:
def BidirModel():
  inpt = Input(shape=(34 , 2))
  x = Bidirectional(GRU(32 , return_sequences=True))(inpt)
  x = Bidirectional(GRU(32 , return_sequences=True))(inpt)
  x = Dense(8, activation="relu")(x)
  x = Dense(2)(x)
  model = Model(inputs=inpt , outputs=x)

  return model

model = BidirModel()
model.compile(loss="mean_squared_error" , optimizer="adam")
model.summary()

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        [(None, 34, 2)]           0         
_________________________________________________________________
bidirectional_40 (Bidirectio (None, 34, 64)            6912      
_________________________________________________________________
dense_64 (Dense)             (None, 34, 8)             520       
_________________________________________________________________
dense_65 (Dense)             (None, 34, 2)             18        
Total params: 7,450
Trainable params: 7,450
Non-trainable params: 0
_________________________________________________________________


In [80]:
model.fit(X_sort, Y_sort, batch_size=4096, epochs= 100)

Epoch 1/100
58/58 [==============================] - 4s 24ms/step - loss: 4.1523
Epoch 2/100
58/58 [==============================] - 1s 22ms/step - loss: 3.9821
Epoch 3/100
58/58 [==============================] - 1s 23ms/step - loss: 3.9356
Epoch 4/100
58/58 [==============================] - 1s 23ms/step - loss: 3.8925
Epoch 5/100
58/58 [==============================] - 1s 23ms/step - loss: 3.8651
Epoch 6/100
58/58 [==============================] - 1s 23ms/step - loss: 3.7838
Epoch 7/100
58/58 [==============================] - 1s 23ms/step - loss: 3.7732
Epoch 8/100
58/58 [==============================] - 1s 23ms/step - loss: 3.7444
Epoch 9/100
58/58 [==============================] - 1s 23ms/step - loss: 3.7492
Epoch 10/100
58/58 [==============================] - 1s 23ms/step - loss: 3.7493
Epoch 11/100
58/58 [==============================] - 1s 23ms/step - loss: 3.7403
Epoch 12/100
58/58 [==============================] - 1s 23ms/step - loss: 3.7365
Epoch 13/100
58/58 [=====

In [78]:
model.evaluate(X_sort_val, Y_sort_val, batch_size=4096)

4/4 [==============================] - 1s 10ms/step - loss: 3.5732


3.573225736618042